# 處理 outliers
* 新增欄位註記
* outliers 或 NA 填補
    1. 平均數 (mean)
    2. 中位數 (median, or Q50)
    3. 最大/最小值 (max/min, Q100, Q0)
    4. 分位數 (quantile)

# [作業目標]
- 仿造範例的資料操作, 試著進行指定的離群值處理

# [作業重點]
- 計算 AMT_ANNUITY 的分位點 (q0 - q100) (Hint : np.percentile, In[3])
- 將 AMT_ANNUITY 的 NaN 用中位數取代 (Hint : q50, In[4])
- 將 AMT_ANNUITY 數值轉換到 -1 ~ 1 之間 (Hint : 參考範例, In[5])
- 將 AMT_GOOD_PRICE 的 NaN 用眾數取代 (In[6])

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# 設定 data_path
dir_data = '../data/'

In [2]:
f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

Path of read in data: ../data/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 列出 AMT_ANNUITY 的 q0 - q100
## 2.1 將 AMT_ANNUITY 中的 NAs 暫時以中位數填補
## 2.2 將 AMT_ANNUITY 的數值標準化至 -1 ~ 1 間
## 3. 將 AMT_GOOD_PRICE 的 NAs 以眾數填補


In [3]:
non_null_amt=app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY']

# 1: 計算 AMT_ANNUITY 的 q0 - q100
q_all =[np.percentile(non_null_amt,q=i) for i in range(101)]

quantile_df=pd.DataFrame({'q': list(range(101)),
              'value': q_all})
quantile_df

,q,value
0,0,1615.50
1,1,6182.91
2,2,6750.00
3,3,7875.00
4,4,8703.00
...,...,...
96,96,55624.50
97,97,58482.00
98,98,62964.00
99,99,70006.50


In [4]:
print(app_train[app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY'])
print(type(q_all),q_all[50])

47531    NaN
50035    NaN
51594    NaN
55025    NaN
59934    NaN
75873    NaN
89343    NaN
123872   NaN
207186   NaN
227939   NaN
239329   NaN
241835   NaN
Name: AMT_ANNUITY, dtype: float64
<class 'list'> 24903.0


In [5]:
# 2.1 將 NAs 以 q50 填補
print("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

"""
Your Code Here
"""

q_50 = q_all[50]
app_train.loc[app_train['AMT_ANNUITY'].isnull(),'AMT_ANNUITY'] = q_50

print("After replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12
After replace NAs, numbers of row that AMT_ANNUITY is NAs: 0


### Hints: Normalize function (to -1 ~ 1)
$ y = 2*(\frac{x - min(x)}{max(x) - min(x)} - 0.5) $

In [8]:
print(type(app_train['AMT_ANNUITY']))
print(app_train['AMT_ANNUITY'].describe())
print(app_train['AMT_ANNUITY'].min())
print(app_train['AMT_ANNUITY'].max())
test_series=app_train['AMT_ANNUITY']*2
    # 對整個series做相同運算
print(test_series)
print(app_train['AMT_ANNUITY'])

<class 'pandas.core.series.Series'>
count    307511.000000
mean      27108.487841
std       14493.461065
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64
1615.5
258025.5
0         49401.0
1         71397.0
2         13500.0
3         59373.0
4         43731.0
           ...   
307506    55116.0
307507    24003.0
307508    59958.0
307509    40410.0
307510    98235.0
Name: AMT_ANNUITY, Length: 307511, dtype: float64
0         24700.5
1         35698.5
2          6750.0
3         29686.5
4         21865.5
           ...   
307506    27558.0
307507    12001.5
307508    29979.0
307509    20205.0
307510    49117.5
Name: AMT_ANNUITY, Length: 307511, dtype: float64


In [9]:
# 2.2 Normalize values to -1 to 1
print("== Original data range ==")
print(app_train['AMT_ANNUITY'].describe())

def normalize_value(x):
    """
    Your Code Here, compelete this function
    """
    x=2*(((x-x.min())/(x.max()-x.min()))-0.5)
    return x

app_train['AMT_ANNUITY_NORMALIZED'] = normalize_value(app_train['AMT_ANNUITY'])
    # normalize_value(series)

print("== Normalized data range ==")
app_train['AMT_ANNUITY_NORMALIZED'].describe()

== Original data range ==
count    307511.000000
mean      27108.487841
std       14493.461065
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64
== Normalized data range ==


count    307511.000000
mean         -0.801154
std           0.113049
min          -1.000000
25%          -0.883714
50%          -0.818357
75%          -0.742752
max           1.000000
Name: AMT_ANNUITY_NORMALIZED, dtype: float64

In [49]:
# 3
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

# 列出重複最多的數值
"""
Your Code Here
"""
appears={}
for k in app_train[~app_train['AMT_GOODS_PRICE'].isnull()]['AMT_GOODS_PRICE']:
    appears[k]=0
for e in app_train[~app_train['AMT_GOODS_PRICE'].isnull()]['AMT_GOODS_PRICE']:
    appears[e]+=1

sorted_pair=sorted(appears.items(),reverse=True,key=lambda d: d[1])
    # 記得用dict.items()
value_most =sorted_pair[0]
print(value_most)
mode_goods_price = list(app_train['AMT_GOODS_PRICE'].value_counts().index)
app_train.loc[app_train['AMT_GOODS_PRICE'].isnull(), 'AMT_GOODS_PRICE'] = mode_goods_price[0]

print("After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))


Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 278
(450000.0, 26022)
After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 0


In [41]:
# 用字典及以字典值來排序找眾數

import time
start_time = time.time()
number_of_key={}
for k in app_train[~app_train['AMT_GOODS_PRICE'].isnull()]['AMT_GOODS_PRICE']:
    number_of_key[k]=0
    # 把所有值的次數都設為0
for e in app_train[~app_train['AMT_GOODS_PRICE'].isnull()]['AMT_GOODS_PRICE']:
    number_of_key[e]+=1
    # 有出現的價格就把次數+1
sorted_pair=sorted(number_of_key.items(),reverse=True,key= lambda d: d[1])
print(sorted_pair[0][0])
    # 用sorted對number_of_key以次數排序，詳見下方
    # 排序完後再回傳次數最高的值即為眾數
print("Elapsed time: %.3f secs" % (time.time() - start_time))

450000.0
Elapsed time: 0.722 secs


# sorted()
    sorted(iterable, reverse=False, key=rule_fn)
        iterable:要排序的串列名稱
        reverse: 是否要反轉，預設是 False, reverse=True 時則代表這個串列要由大到小排
        key: 就是指我們所要用來排序的規則。

In [30]:
dic = {"2017": "The Shape of Water", "2016": "Moonlight", "2014": "Birdman", "2015": "Spotlight"}
sorted(dic)
    # 直接對字典排序會對key排序

['2014', '2015', '2016', '2017']

In [31]:
sorted(dic.values())
    # 只對字典的值排序

['Birdman', 'Moonlight', 'Spotlight', 'The Shape of Water']

In [37]:
'''
https://medium.com/ccclub/ccclub-python-for-beginners-tutorial-f3148ebb33a4
寫在 lambda 後面的 d 是這個 function 的參數，而在冒號 : 後方的則是這個 function 的回傳值，
在這裡是 d[1] ，意思即是當這個 key function 每次拿到一個 tuple 作為參數，就取出 index 為 1 的值回傳。
'''
sorted(dic.items(), key = lambda d: d[1])
    # 以value值進行排序, key為自訂的function

[('2014', 'Birdman'),
 ('2016', 'Moonlight'),
 ('2015', 'Spotlight'),
 ('2017', 'The Shape of Water')]

In [47]:
dic.items()

dict_items([('2017', 'The Shape of Water'), ('2016', 'Moonlight'), ('2014', 'Birdman'), ('2015', 'Spotlight')])